In [44]:
# Import required libraries
import networkx as nx
import numpy as np
from scipy.special import factorial
from datetime import datetime
import logging
import sys

In [45]:
import logging

# Define the output log file
outputLogFile = "experiment_1.log"

# Configure the logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.handlers = []  # Clear any existing handlers

# Create a file handler that appends to the log file
file_handler = logging.FileHandler(outputLogFile, mode='a')
file_handler.setLevel(logging.INFO)

# Set a simple log format that just prints the message
formatter = logging.Formatter('%(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# Ensure logs are flushed to file after each write
for handler in logger.handlers:
    handler.flush()

logger.propagate = False

logger.info("Logging initialized. Will append to the existing experiment_1.log file.")

In [46]:
# Log start of experiment
logger.info(f'[1] "experiment_1.log"')
logger.info(f'[1] "{datetime.now().strftime("%a %b %d %H:%M:%S %Y")}"')

In [47]:
%run attack_graph_MARA.ipynb

In [51]:
# Define parameters (like attackRateList in R)
attack_rate_list = [2]  
defense_rate_list = [0]

# Define random_steps function (like in R)
def random_steps(route, attack_rate=None, defense_rate=None):
    length = len(route)
    if attack_rate is None:
        attack_rate = 2
    x = np.arange(length)
    pmf = (np.power(attack_rate, x) * np.exp(-attack_rate)) / np.vectorize(factorial)(x)
    pmf = pmf / pmf.sum()
    return pmf

In [49]:
# Import and run the main Cut-The-Rope implementation
%run ctr-core_1.ipynb

DEBUG: Roots before adding entry node: [1]
DEBUG: Only one root found. Entry node is: 1
DEBUG: Current graph nodes after handling entry node: [1, 2, 3, 4, 5, 6, 7, 8, 9]
DEBUG: Current graph edges after handling entry node:
  1 -> 2
  2 -> 3
  2 -> 4
  2 -> 5
  3 -> 6
  4 -> 6
  5 -> 7
  7 -> 8
  8 -> 9
DEBUG: Final graph after processing targets:
Nodes: [1, 2, 3, 4, 5, 8, 6, 7]
Edges:
  1 -> 2
  2 -> 3
  2 -> 4
  2 -> 5
  3 -> 8
  4 -> 8
  5 -> 6
  6 -> 7
  7 -> 8
DEBUG: New targets after merging: [8]


In [50]:
# Read and display log file contents
with open('experiment_1.log', 'r') as f:
    print(f.read())

Logging initialized. Will append to the existing experiment_1.log file.
[1] "experiment_1.log"
[1] "Sun Dec 08 18:18:14 2024"

++++++++++++++++++++++++++++++++
attack rate =  2 , defense rate =  0 

	equilibrium for multiobjective security game (MOSG)

optimal defense strategy:
         prob.
2 0.000000e+00
3 2.798527e-01
4 2.798527e-01
5 0.000000e+00
6 0.000000e+00
7 4.402946e-01

worst case attack strategies per goal:
          1
1 0.1280262
[1] 0.128

++++++++++++++++++++++++++++++++
attack rate =  3 , defense rate =  0 

	equilibrium for multiobjective security game (MOSG)

optimal defense strategy:
         prob.
2 0.000000e+00
3 2.588458e-01
4 2.588458e-01
5 0.000000e+00
6 0.000000e+00
7 4.823085e-01

worst case attack strategies per goal:
          1
1 0.1580403
[1] 0.158

++++++++++++++++++++++++++++++++
attack rate =  2 , defense rate =  1 

	equilibrium for multiobjective security game (MOSG)

optimal defense strategy:
         prob.
2 0.000000e+00
3 2.798527e-01
4 2.798527e-